In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df=pd.read_csv('C:/Users/aenor/Desktop/recom/temptest.csv', encoding = 'utf-8')

data = df.pivot_table('rating', index = 'userid', columns = 'appid')
# print(data.head())

ratings = pd.read_csv("C:/Users/aenor/Desktop/recom/resultq.csv", encoding = 'utf-8')
games = pd.read_csv( 'C:/Users/aenor/Desktop/recom/temptest.csv', encoding ='utf-8')


In [4]:
ratings = pd.read_csv("C:/Users/aenor/Desktop/recom/resultq.csv", encoding = 'utf-8')
ratings=ratings.drop(columns=['english','release_date','developer','publisher','platforms','required_age','categories','steamspy_tags','achievements','positive_ratings','negative_ratings','average_playtime','median_playtime','owners','price'])
ratings

,appid,name,genres
0,1,Counter-Strike,Action
1,2,Team Fortress Classic,Action
2,3,Day of Defeat,Action
3,4,Half-Life: Opposing Force,Action
4,5,Ricochet,Action
...,...,...,...
1625,1626,Hentai Girl Betty,Casual;Indie;RPG;Simulation
1626,1627,DOKA 2 KISHKI EDITION,Action;Adventure;Casual;Indie;RPG;Simulation
1627,1628,Thronebreaker: The Witcher Tales,Adventure;RPG
1628,1629,Wet Girl,Adventure;Casual;Indie;RPG


In [5]:
ratings_games = pd.merge(ratings , games , on = 'appid')
ratings_games


,appid,name,genres,userid,rating
0,1,Counter-Strike,Action,7,0
1,1,Counter-Strike,Action,10,0
2,1,Counter-Strike,Action,24,1
3,1,Counter-Strike,Action,29,1
4,1,Counter-Strike,Action,41,0
...,...,...,...,...,...
1327,301,Hell Yeah! Wrath of the Dead Rabbit,Action,59,0
1328,302,Tiny and Big: Grandpa's Leftovers,Indie,0,0
1329,302,Tiny and Big: Grandpa's Leftovers,Indie,7,1
1330,302,Tiny and Big: Grandpa's Leftovers,Indie,85,1


In [6]:
ratings_games.groupby('appid')

In [7]:
filter=ratings_games['appid']==50
ratings_games[filter]['name'].unique()

array(['Warhammer嶺?40,000: Dawn of War嶺?- Dark Crusade'], dtype=object)

In [8]:
####Buillding a Utility Matrix####
rating_crosstab=ratings_games.pivot_table(values='rating',index='userid',columns='name',fill_value=0)
rating_crosstab

name,APB Reloaded,ARMA: Cold War Assault,Age of Empires嶺?III: Complete Collection,Alan Wake,Alan Wake's American Nightmare,Alien Swarm,Altitude,America's Army 3,America's Army: Proving Grounds,Amnesia: The Dark Descent,...,Worms Revolution,Worms Ultimate Mayhem,X Rebirth,X-Blades,X-COM: UFO Defense,X3: Terran Conflict,XCOM: Enemy Unknown,Zeno Clash,Zombie Panic! Source,ibb & obb
userid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#transpose
X=rating_crosstab.T
X

userid,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
name,,,,,,,,,,,,,,,,,,,,,
APB Reloaded,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ARMA: Cold War Assault,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age of Empires嶺?III: Complete Collection,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Alan Wake,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alan Wake's American Nightmare,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X3: Terran Conflict,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
XCOM: Enemy Unknown,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Zeno Clash,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#Decomposing the Matrix
SVD=TruncatedSVD(n_components=12,random_state=17)
resultant_matrix=SVD.fit_transform(X)
resultant_matrix.shape

(301, 12)

In [11]:
#Generating a Correlation Matrix
corr_mat=np.corrcoef(resultant_matrix)
corr_mat.shape

c:\Users\aenor\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\Users\aenor\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


(301, 301)

In [24]:
#Isolating 'ARMA: Cold War Assault' From the Correlation Matrix
game_names=rating_crosstab.columns
game_list=list(game_names)
The_game=game_list.index('ARMA: Cold War Assault')
The_game

1

In [30]:
cor=corr_mat[The_game]
cor.shape

(301,)

In [45]:
cor_game=list(game_names[(cor<1)&(cor>0.5)])
cor_num=list(cor[(cor<1)&(cor>0.5)])
result_mat=[]
result_mat.append(cor_game)
result_mat.append(cor_num)
result=pd.DataFrame(result_mat).transpose()
result.columns=['name','simularity']
result=result.sort_values(by='simularity',ascending=False)
result

,name,simularity
5,MX vs. ATV Reflex,0.967759
7,ORION: Prelude,0.811547
10,Superbrothers: Sword & Sworcery EP,0.729037
9,SimCity??4 Deluxe Edition,0.717004
0,Amnesia: The Dark Descent,0.714934
3,F.E.A.R. 3,0.669899
6,Moonbase Alpha,0.639
2,Call of Duty嶺? Black Ops,0.62275
1,Assassin's Creed嶺?Revelations,0.520138
8,QUAKE,0.520138
